[![Open In Collab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/langchain-ai/langchain/blob/master/docs/docs/use_cases/extraction.ipynb)

## Use case

LLMs can be used to extract information from text. 

There are 3 broad approaches for information extraction using LLMs:

- **Schema Based**: There are LLMs can generate output according to a provided schema. These LLMs are often used to invoke **tools**, so this mode is commonly called **"function-calling"** or **"tool-calling"**. For example, see [OpenAI's function and tool calling](https://platform.openai.com/docs/guides/function-calling)

- **JSON Mode**: LLMs are forced to output valid JSON. Similar to **Schema Based** approach, except the schema is provided as part of the prompt (i.e., in text). Generally, expected to perform worse than a **Schema Based** approach. For example, see [OpenAI's JSON mode](https://platform.openai.com/docs/guides/text-generation/json-mode).

- **Prompt Based**: LLMs that can follow instructions well can be instructed to generate text in a desired format. This text can then be pased using existing [Output Parsers](/docs/modules/model_io/output_parsers/) or [custom parsers](/docs/modules/model_io/output_parsers/custom) into structured format (e.g., JSON).

## Option 1: Schema Based Generation (aka tool-calling)

### Quickstart

Use `create_structured_output_runnable` to create a LangChain `Runnable` that supports information extraction using **tool calling**.

The desired output schema can be expressed either via a `Pydantic` model or a Python dict representing OpenAI's schema for tool calling (very similar to [JsonSchema](https://json-schema.org/)).

`create_structured_output_runnable` supports both `openai-tools` and `openai-functions` formats. 

:::{.callout-tip}
Model providers like [Fireworks](https://fireworks.ai/) use the same format specification as OpenAI. So the formats `openai-tools` and `openai-functions` can be used with other model providers as well if they support them.
:::

In [1]:
pip install langchain langchain-openai 

# Set env var OPENAI_API_KEY or load from a .env file:
# import dotenv
# dotenv.load_dotenv()

SyntaxError: invalid syntax (2907515574.py, line 1)

### Specifying Schemas

There are two best practices when defining schema for a model:

1. **Document** the attributes and model: This information is sent to the model and is used to improve the quality of information extraction.
2. Build in an **"escape-hatch"** into the schema that allows the model to correctly **REFUSE** to extract information if no relevant information can be extracted.

Let's define an example schema to extract some information about a person.

In [64]:
from langchain_core.pydantic_v1 import BaseModel, Field

class Person(BaseModel):
    """Information about a person.""" # <-- Doc-string for the entity Person
    # Below we define two **optional** fields called `name` and `hair_color`.
    # The model is allowed to provide a `null` if no relevant information is present in the text.
    # In addition, the fields are documented using a `description`.
    # Feel free to make the description more specific (e.g., to include an example of hair color etc.)
    name: str = Field(..., description="The name of the person")
    hair_color: str = Field(..., description="The color of the peron's eyes if known")

:::{.callout-tip} Document the schema!

Schema documentation is sent to the LLM to help it extract information correctly!

* Document all attributes using a **description**.
* Document the model itself using a doc-string under the class.
:::

Test out the extraction

In [2]:
from typing import Optional
from langchain.chains import create_structured_output_runnable
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-4-0125-preview",
    temperature=0,
)

In [3]:
text = "Bobby Smith is 5 feet tall and has blond hair."
runnable.invoke({'text': text})

NameError: name 'runnable' is not defined

#### Escape Hatches

**"Escape hatches"** allow the model to extract no information if there's no information to be extracted.

If you're working on a problem where you need to extract information from across a collection of documents, providing these escape hatches will become extremely important

It's extremely important to allow the model to decline to extract information if the information is not present in the content, especially if extracting information from many document


When doing extraction, it's often helpful to mark the fields of the model as **optional** to allow

In [61]:
text = "Bobby Smith is 5 feet tall and has blond hair."
runnable.invoke({'text': text})

Person(name='Bobby Smith', hair_color='blond')

In [46]:
from langchain_core.prompts import ChatPromptTemplate

In [15]:
prompt = ChatPromptTemplate.from_messages(
    [('system', 'You are an expert extraction algorithm. '
    'Only extract relevant information from the text. '
    'Do not extract information that does not match the description'),
     ('human', '{text}')]
     )

In [16]:
runnable = create_structured_output_runnable(Person, llm, prompt=prompt)

In [19]:
text = "Earth has only 1 moon."
runnable.invoke({'text': text})

Person(name=None, hair_color=None)

### Specifying schemas

A convenient way to express desired output schemas is via Pydantic. The above example specified the desired output schema via `Person`, a Pydantic model. Such schemas can be easily combined together to generate richer output formats:

In [2]:
from typing import Sequence


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person]


runnable = create_structured_output_runnable(People, llm)

In [ ]:
inp = """Alex is 5 feet tall and has blond hair.
Claudia is 1 feet taller Alex and jumps higher than him.
Claudia is a brunette and has a beagle named Harry."""

runnable.invoke(inp)

People(people=[Person(person_name='Alex', person_height=5, person_hair_color='blond', dog_breed=None, dog_name=None), Person(person_name='Claudia', person_height=6, person_hair_color='brunette', dog_breed='beagle', dog_name='Harry')])

Note that `dog_breed` and `dog_name` are optional attributes, such that here they are extracted for Claudia and not for Alex.

One can also specify the desired output format with a Python dict representing valid JsonSchema:

In [ ]:
schema = {
    "type": "object",
    "properties": {
        "name": {"type": "string"},
        "height": {"type": "integer"},
        "hair_color": {"type": "string"},
    },
    "required": ["name", "height"],
}

runnable = create_structured_output_runnable(schema, llm)

In [ ]:
inp = "Alex is 5 feet tall. I don't know his hair color."
runnable.invoke(inp)

{'name': 'Alex', 'height': 60}

In [ ]:
inp = "Alex is 5 feet tall. He is blond."
runnable.invoke(inp)

{'name': 'Alex', 'height': 60, 'hair_color': 'blond'}

#### Extra information

Runnables constructed via `create_structured_output_runnable` generally are capable of semantic extraction, such that they can populate information that is not explicitly enumerated in the schema.

Suppose we want unspecified additional information about dogs. 

We can use add a placeholder for unstructured extraction, `dog_extra_info`.

In [3]:
inp = """Alex is 5 feet tall and has blond hair.
Claudia is 1 feet taller Alex and jumps higher than him.
Claudia is a brunette and has a beagle named Harry.
Harry likes to play with other dogs and can always be found
playing with Milo, a border collie that lives close by."""

In [4]:
class Person(BaseModel):
    person_name: str
    person_height: int
    person_hair_color: str
    dog_breed: Optional[str]
    dog_name: Optional[str]
    dog_extra_info: Optional[str]


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person]


runnable = create_structured_output_runnable(People, llm)
runnable.invoke(inp)

People(people=[Person(person_name='Alex', person_height=60, person_hair_color='blond', dog_breed=None, dog_name=None, dog_extra_info=None), Person(person_name='Claudia', person_height=66, person_hair_color='brunette', dog_breed='beagle', dog_name='Harry', dog_extra_info='likes to play with other dogs, especially with Milo, a border collie')])

This gives us additional information about the dogs.

### Specifying extraction mode

`create_structured_output_runnable` supports varying implementations of the underlying extraction under the hood, which are configured via the `mode` parameter. This parameter can be one of `"openai-functions"`, `"openai-tools"`, or `"openai-json"`.

#### OpenAI Functions and Tools

Some LLMs are fine-tuned to support the invocation of functions or tools. If they are given an input schema for a tool and recognize an occasion to use it, they may emit JSON output conforming to that schema. We can leverage this to drive structured data extraction from natural language.

OpenAI originally released this via a [`functions` parameter in its chat completions API](https://openai.com/blog/function-calling-and-other-api-updates). This has since been deprecated in favor of a [`tools` parameter](https://platform.openai.com/docs/guides/function-calling), which can include (multiple) functions.

Using OpenAI Functions:

In [1]:
runnable = create_structured_output_runnable(Person, llm, mode="openai-functions")

text = "Alex is 5 feet tall and has blond hair."
runnable.invoke(text)

NameError: name 'create_structured_output_runnable' is not defined

Using OpenAI Tools:

In [ ]:
runnable = create_structured_output_runnable(Person, llm, mode="openai-tools")

runnable.invoke(inp)

Person(person_name='Alex', person_height=152, person_hair_color='blond', dog_breed=None, dog_name=None)

The corresponding [LangSmith trace](https://smith.langchain.com/public/04cc37a7-7a1c-4bae-b972-1cb1a642568c/r) illustrates the tool call that generated our structured output.

![Image description](../../static/img/extraction_trace_tool.png)

#### JSON Mode

Some LLMs support generating JSON more generally. OpenAI implements this via a [`response_format` parameter](https://platform.openai.com/docs/guides/text-generation/json-mode) in its chat completions API.

Note that this method may require explicit prompting (e.g., OpenAI requires that input messages contain the word "json" in some form when using this parameter).

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

system_prompt = """You extract information in structured JSON formats.

Extract a valid JSON blob from the user input that matches the following JSON Schema:

{output_schema}"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
runnable = create_structured_output_runnable(
    Person,
    llm,
    mode="openai-json",
    prompt=prompt,
    enforce_function_usage=False,
)

runnable.invoke({"input": inp})

Person(person_name='Alex', person_height=5, person_hair_color='blond', dog_breed=None, dog_name=None, dog_extra_info=None)

### Few-shot examples

Suppose we want to tune the behavior of our extractor. There are a few options available. For example, if we want to redact names but retain other information, we could adjust the system prompt:

In [ ]:
system_prompt = """You extract information in structured JSON formats.

Extract a valid JSON blob from the user input that matches the following JSON Schema:

{output_schema}

Redact all names.
"""
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), ("human", "{input}")]
)
runnable = create_structured_output_runnable(
    Person,
    llm,
    mode="openai-json",
    prompt=prompt,
    enforce_function_usage=False,
)

runnable.invoke({"input": inp})

Person(person_name='REDACTED', person_height=5, person_hair_color='blond', dog_breed=None, dog_name=None)

Few-shot examples are another, effective way to illustrate intended behavior. For instance, if we want to redact names with a specific character string, a one-shot example will convey this. We can use a `FewShotChatMessagePromptTemplate` to easily accommodate both a fixed set of examples as well as the dynamic selection of examples based on the input.

In [ ]:
from langchain_core.prompts import FewShotChatMessagePromptTemplate

examples = [
    {
        "input": "Samus is 6 ft tall and blonde.",
        "output": Person(
            person_name="######",
            person_height=5,
            person_hair_color="blonde",
        ).dict(),
    }
]

example_prompt = ChatPromptTemplate.from_messages(
    [("human", "{input}"), ("ai", "{output}")]
)
few_shot_prompt = FewShotChatMessagePromptTemplate(
    examples=examples,
    example_prompt=example_prompt,
)
prompt = ChatPromptTemplate.from_messages(
    [("system", system_prompt), few_shot_prompt, ("human", "{input}")]
)
runnable = create_structured_output_runnable(
    Person,
    llm,
    mode="openai-json",
    prompt=prompt,
    enforce_function_usage=False,
)

runnable.invoke({"input": inp})

Person(person_name='#####', person_height=5, person_hair_color='blond', dog_breed=None, dog_name=None)

Here, the [LangSmith trace](https://smith.langchain.com/public/6fe5e694-9c04-48f7-83ff-e541da764781/r) for the chat model call shows how the one-shot example is formatted into the prompt.

![Image description](../../static/img/extraction_trace_few_shot.png)

## Option 2: Parsing

[Output parsers](/docs/modules/model_io/output_parsers/) are classes that help structure language model responses. 

As shown above, they are used to parse the output of the runnable created by `create_structured_output_runnable`.

They can also be used more generally, if a LLM is instructed to emit its output in a certain format. Parsers include convenience methods for generating formatting instructions for use in prompts.

Below we implement an example.

In [ ]:
from typing import Optional, Sequence

from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain_core.pydantic_v1 import BaseModel, Field, validator
from langchain_openai import ChatOpenAI


class Person(BaseModel):
    person_name: str
    person_height: int
    person_hair_color: str
    dog_breed: Optional[str]
    dog_name: Optional[str]


class People(BaseModel):
    """Identifying information about all people in a text."""

    people: Sequence[Person]


# Run
query = """Alex is 5 feet tall. Claudia is 1 feet taller Alex and jumps higher than him. Claudia is a brunette and Alex is blond."""

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=People)

# Prompt
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Run
_input = prompt.format_prompt(query=query)
model = ChatOpenAI()

In [ ]:
output = model.invoke(_input.to_string())

parser.parse(output.content)

People(people=[Person(person_name='Alex', person_height=5, person_hair_color='blond', dog_breed=None, dog_name=None), Person(person_name='Claudia', person_height=6, person_hair_color='brunette', dog_breed=None, dog_name=None)])

We can see from the [LangSmith trace](https://smith.langchain.com/public/aec42dd3-d471-4d34-801b-20dd88444931/r) that we get the same output as above.

![Image description](../../static/img/extraction_trace_parsing.png)

We can see that we provide a two-shot prompt in order to instruct the LLM to output in our desired format.

In [ ]:
# Define your desired data structure.
class Joke(BaseModel):
    setup: str = Field(description="question to set up a joke")
    punchline: str = Field(description="answer to resolve the joke")

    # You can add custom validation logic easily with Pydantic.
    @validator("setup")
    def question_ends_with_question_mark(cls, field):
        if field[-1] != "?":
            raise ValueError("Badly formed question!")
        return field


# And a query intended to prompt a language model to populate the data structure.
joke_query = "Tell me a joke."

# Set up a parser + inject instructions into the prompt template.
parser = PydanticOutputParser(pydantic_object=Joke)

# Prompt
prompt = PromptTemplate(
    template="Answer the user query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

# Run
_input = prompt.format_prompt(query=joke_query)
model = ChatOpenAI(temperature=0)
output = model.invoke(_input.to_string())
parser.parse(output.content)

Joke(setup="Why couldn't the bicycle find its way home?", punchline='Because it lost its bearings!')

As we can see, we get an output of the `Joke` class, which respects our originally desired schema: 'setup' and 'punchline'.

We can look at the [LangSmith trace](https://smith.langchain.com/public/557ad630-af35-43e9-b043-93800539025f/r) to see exactly what is going on under the hood.

### Going deeper

* The [output parser](/docs/modules/model_io/output_parsers/) documentation includes various parser examples for specific types (e.g., lists, datetime, enum, etc).  
* The experimental [Anthropic function calling](https://python.langchain.com/docs/integrations/chat/anthropic_functions) support provides similar functionality to Anthropic chat models.
* [LlamaCPP](https://python.langchain.com/docs/integrations/llms/llamacpp#grammars) natively supports constrained decoding using custom grammars, making it easy to output structured content using local LLMs 
* [JSONFormer](/docs/integrations/llms/jsonformer_experimental) offers another way for structured decoding of a subset of the JSON Schema.
* [Kor](https://eyurtsev.github.io/kor/) is another library for extraction where schema and examples can be provided to the LLM.

## Extracting from different input formats

There are many types of data we may wish to extract from. We will look into examples of extracting information from binary files and longer documents that do not fit into the model context window.

### Longer Documents

In the case of extracting from a very long text file, we want to split the content using langchain provided tools and then deduplicate the data from the resulting extractions.

Below we implement an example:

In [5]:
from langchain.text_splitter import TokenTextSplitter

with open(r"./test.txt", "r") as fp:
    doc_content = fp.read().strip()

runnable = create_structured_output_runnable(People, llm, mode="openai-tools")

text_splitter = TokenTextSplitter(
    chunk_size=1000,
    chunk_overlap=20,
)

texts = text_splitter.split_text(doc_content)
extract_responses = runnable.batch(texts)
dedup_responses = []
for response in extract_responses:
    for person in response.people:
        if person not in dedup_responses:
            dedup_responses.append(person)

In [ ]:
### Binary Files

Besides raw text data, you may wish to extract information from other file types such as powerpoints or pdfs. To do so, we can use the Blob object along with a PDF parser.

A code example is below:

from langchain.document_loaders.parsers import PDFMinerParser
from langchain_community.document_loaders import Blob

with open(r"./test.pdf", "rb") as fp:
    file_data = fp.read()

blob = Blob.from_data(
    data=file_data,
    mime_type="application/pdf",
)

parser = PDFMinerParser()
parsed_data = parser.parse(blob=blob)
text = parsed_data[0].page_content.strip()

runnable = create_structured_output_runnable(People, llm, mode="openai-tools")
runnable.invoke(text)

### Mime Type Based Parsing

In addition, you can choose to not specify the file and parser type directly, instead letting the code infer it from the file it gets passed.

A code example of this is below:

import magic
from langchain.document_loaders.parsers import BS4HTMLParser, PDFMinerParser
from langchain.document_loaders.parsers.generic import MimeTypeBasedParser
from langchain.document_loaders.parsers.txt import TextParser

HANDLERS = {
    "application/pdf": PDFMinerParser(),
    "text/plain": TextParser(),
    "text/html": BS4HTMLParser(),
}

MIMETYPE_BASED_PARSER = MimeTypeBasedParser(
    handlers=HANDLERS,
    fallback_parser=None,
)

with open(r"./test.html", "rb") as fp:
    file_data = fp.read()

mime = magic.Magic(mime=True)
mime_type = mime.from_buffer(file_data)

blob = Blob.from_data(
    data=file_data,
    mime_type=mime_type,
)

parser = HANDLERS[mime_type]
parsed_data = parser.parse(blob=blob)
runnable.invoke(parsed_data[0].page_content)